In [6]:
import os
import math
import numpy as np
from DatosLogin import login
from Conectores import conectorMSSQL
from PIL import Image
import pandas as pd
from pandas.api.types import CategoricalDtype
import dataframe_image as dfi
import pyodbc #Library to connect to Microsoft SQL Server
from DatosLogin import login #Holds connection data to the SQL Server
import sys
import pathlib
from datetime import datetime
from datetime import timedelta
#sys.path.insert(0,str(pathlib.Path(__file__).parent.parent))
import logging
logging.basicConfig(
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
        , level=logging.INFO
)
logger = logging.getLogger(__name__)

#################################

tiempoInicio = pd.to_datetime("today")
#########
# Formating display of dataframes with comma separator for numbers
pd.options.display.float_format = "{:20,.0f}".format 
#########


try:
    db_conex = pyodbc.connect(
        "DRIVER={ODBC Driver 17 for SQL Server};\
        SERVER="+login[0]+";\
        DATABASE="+login[1]+";\
        UID="+login[2]+";\
        PWD="+ login[3]
    )
except Exception as e:
    listaErrores = e.args[1].split(".")
    logger.error("\nOcurrió un error al conectar a SQL Server: ")
    for i in listaErrores:
        logger.error(i)
    exit()



vtasSalon = pd.read_sql('''
	
    SELECT A.UEN, COUNT(DISTINCT A.DESPACHOS) AS ID, SUM(A.IMPORTE) AS 'TOTAL VENDIDO' FROM(
    SELECT ID AS DESPACHOS, UEN, IMPORTE
    FROM [Rumaos].[dbo].[SCEgreso] where
    FECHASQL = DATEADD(day, -1, CAST(GETDATE() AS date))
    ) AS A GROUP BY  UEN

    ''' ,db_conex)


vtasSalon = vtasSalon.convert_dtypes()
vtasSalon = vtasSalon.reset_index()


vtasSalon = vtasSalon.assign(Promedio= vtasSalon['TOTAL VENDIDO'] / vtasSalon['ID'])

vtasSalonM = pd.read_sql('''
    DECLARE @ayer DATETIME
    SET @ayer = DATEADD(day, -1, CAST(GETDATE() AS date))
    DECLARE @inicioMesActual DATETIME
	SET @inicioMesActual = DATEADD(month, DATEDIFF(month, 0, @ayer), 0)

	DECLARE @inicioMesAnterior DATETIME
	SET @inicioMesAnterior = DATEADD(M,-1,@inicioMesActual)

	--Divide por la cant de días del mes anterior y multiplica por la cant de días del
	--mes actual
	
	DECLARE @hoy DATETIME
	SET @hoy = DATEADD(DAY, DATEDIFF(DAY, 0, CURRENT_TIMESTAMP), 0)

	--Divide por la cantidad de días cursados del mes actual y multiplica por la cant
	--de días del mes actual
	
	SELECT *
    FROM [Rumaos].[dbo].[SCEgreso] where
  	FECHASQL >= @inicioMesActual
	and	FECHASQL < @hoy

    ''' ,db_conex)
vtasSalonM = vtasSalonM.convert_dtypes()
vtasSalonM = vtasSalonM.reset_index()


C:\Users\mmagistretti\AppData\Local\Temp\ipykernel_16092\1844373468.py:50: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vtasSalon = pd.read_sql('''
C:\Users\mmagistretti\AppData\Local\Temp\ipykernel_16092\1844373468.py:67: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vtasSalonM = pd.read_sql('''


In [8]:

for e in vtasSalonM['CODIGO'].unique():
    vtasSalonM['PREUNIT'] = vtasSalonM['PREUNIT'].fillna(method='ffill')


In [9]:
vtasSalonM

,index,ID,UEN,FECHASQL,TURNO,CODIGO,CANTIDAD,PREUNIT,IMPORTE,FECHAMOVIMSQL,ANULADO,CANTANU,CodVendedor,PreCostoImpIncl,IDCodigoUnico
0,0,172552,AZCUENAGA,2023-10-01,1,7790315000439,1,550,550,2023-10-01 06:33:50.090,0,0,0,211,08300152000000216535001
1,1,172553,AZCUENAGA,2023-10-01,1,7790895006029,1,740,740,2023-10-01 06:33:50.150,0,0,0,381,08300152000000216535002
2,2,172554,AZCUENAGA,2023-10-01,1,32144,1,"2,040","2,040",2023-10-01 06:33:50.220,0,0,0,"1,254",08300152000000216535003
3,3,172555,AZCUENAGA,2023-10-01,1,77903518,1,465,465,2023-10-01 06:33:50.300,0,0,0,223,08300152000000216535004
4,4,172556,AZCUENAGA,2023-10-01,1,1317,2,"1,300","2,600",2023-10-01 06:42:24.680,0,0,0,657,08300152000000216536001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32618,32618,3365014,PERDRIEL,2023-10-10,3,77939753,1,310,310,2023-10-11 05:49:31.140,0,0,<NA>,139,08300125000000098133001
32619,32619,3365015,PERDRIEL,2023-10-10,3,77904744,1,160,160,2023-10-11 05:49:31.980,0,0,<NA>,42,08300125000000098133002
32620,32620,3365016,PERDRIEL,2023-10-10,3,7790580602000,1,235,235,2023-10-11 05:49:32.310,0,0,<NA>,89,08300125000000098133003
32621,32621,3365017,PERDRIEL,2023-10-10,3,78605831,1,220,220,2023-10-11 05:49:32.860,0,0,<NA>,96,08300125000000098133004


In [ ]:


def _estiladorVtaTituloP(df,list_Col_Num0, list_Col_Num, list_Col_Perc, titulo):
    """
This function will return a styled dataframe that must be assign to a variable.
ARGS:
    df: Dataframe that will be styled.
    list_Col_Num: List of numeric columns that will be formatted with
    zero decimals and thousand separator.
    list_Col_Perc: List of numeric columns that will be formatted 
    as percentage.
    titulo: String for the table caption.
    """
    resultado = df.style \
        .format("{0:,.0f}", subset=list_Col_Num0) \
        .format("{0:,.2f}", subset=list_Col_Num) \
        .format("{:,.2%}", subset=list_Col_Perc) \
        .hide_index() \
        .set_caption(
            titulo
            + "<br>"
            + ((tiempoInicio-pd.to_timedelta(1,"days")).strftime("%d/%m/%y"))
            + "<br>") \
        .set_properties(subset= list_Col_Perc + list_Col_Num + list_Col_Num0
            , **{"text-align": "center", "width": "100px"}) \
        .set_properties(border= "2px solid black") \
        .set_table_styles([
            {"selector": "caption", 
                "props": [
                    ("font-size", "20px")
                    ,("text-align", "center")
                ]
            }
            , {"selector": "th", 
                "props": [
                    ("text-align", "center")
                    ,("background-color","black")
                    ,("color","white")
                ]
            }
        ]) \
        .apply(lambda x: ["background: black" if x.name == "colTOTAL" 
            else "" for i in x]
            , axis=1) \
        .apply(lambda x: ["color: white" if x.name == "colTOTAL" 
            else "" for i in x]
            , axis=1)
    evitarTotales = df.index.get_level_values(0)
    resultado = resultado.background_gradient(
        cmap="Dark2" # Red->Yellow->Green
        ,vmin=0.001
        ,vmax=0.007
        ,subset=pd.IndexSlice["Promedio Ticket Diario $"]
    )
    resultado = resultado.background_gradient(
        cmap="Dark2" # Red->Yellow->Green
        ,vmin=0.001
        ,vmax=0.007
        ,subset=pd.IndexSlice["Promedio Ticket Acumulado $"]
    )
    resultado = resultado.background_gradient(
        cmap="RdYlGn" # Red->Yellow->Green
        ,vmin=-0.01
        ,vmax=0.01
        ,subset=pd.IndexSlice['Desvio %']
    )
    return resultado

numCols0 = ["Total Vendido Diario $"
        ,"Total Vendido Acumulado $"
        ,"Tickets Diario"
        ,"Tickets Acumulado"

]

numCols = [ "Promedio Ticket Diario $"
        , "Promedio Ticket Acumulado $"
        ,"Objetivo"
         ]

percColsPen = ['Desvio %'
]


vtasSalonT = _estiladorVtaTituloP(vtasSalonT,numCols0,numCols,percColsPen, "INFO TICKET SALON PROMEDIO")

ubicacion = "C:/Informes/Penetracion Salon/"
nombrePen = "Info_Promedio_Vtas.png"

def df_to_image(df, ubicacion, nombre):
    """
    Esta función usa las biblioteca "dataframe_Image as dfi" y "os" para 
    generar un archivo .png de un dataframe. Si el archivo ya existe, este será
    reemplazado por el nuevo archivo.

    Args:
        df: dataframe a convertir
         ubicacion: ubicacion local donde se quiere grabar el archivo
          nombre: nombre del archivo incluyendo extensión .png (ej: "hello.png")

    """
    if os.path.exists(ubicacion+nombre):
        os.remove(ubicacion+nombre)
        dfi.export(df, ubicacion+nombre)
    else:
        dfi.export(df, ubicacion+nombre)

df_to_image(vtasSalonT, ubicacion, nombrePen)
#df_to_image(vtasSalonM,ubicacion,nombrePenDiario)


#############
# Timer
tiempoFinal = pd.to_datetime("today")
logger.info(
    "\nInfo Volumen de Ventas"
    + "\nTiempo de Ejecucion Total: "
    + str(tiempoFinal-tiempoInicio)
)
